#### -----------------------------------------------------------------------------<br>Copyright (c) 2022, Lucid Vision Labs, Inc.
##### THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN  THE  SOFTWARE.<br>-----------------------------------------------------------------------------

In [ ]:
import time
from datetime import datetime

from arena_api.callback import callback, callback_function
from arena_api.system import system

#### Callback: Image Callbacks
>    This example demonstrates configuring an image callback for a device. Once
    a callback is registered and the device is streaming, the user-implemented
    print_buffer function will be called. print_buffer will receive the buffer
    with the image and will display the frame id and timestamp of the
    image before returning.

In [ ]:
TAB1 = "  "
TAB2 = "    "

In [ ]:
"""
Waits for the user to connect a device before raising an exception
if it fails
"""

tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                    '.' * sec_count, end='\r')
        tries += 1
    else:
        print(f'Created {len(devices)} device(s)')
        break

else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')

##### Must have the decorator on the callback function
> device.on_buffer decorator requires buffer as its first parameter<br>
Buffer should only be accessed by a single thread at a time

In [ ]:
@callback_function.device.on_buffer
def print_buffer(buffer, *args, **kwargs):
    
    now = kwargs['now_func']()
    print(f'{TAB2}Buffer: [{buffer.width} X {buffer.height}] pixels, '
          f'TimeStamp: [{now}]')

##### Demonstrates callback on buffer
> - Connect device, setup stream
> - Register the callback function using callback.register
> - Start the stream
> - As the buffers on the device get filled, the callback is triggered
> - Stop stream and deregister the callback before destroying the device

In [ ]:
device = devices[0]
print(f'{TAB1}Device used in the example:\n{TAB1}{device}')

##### Setup stream values

In [ ]:
tl_stream_nodemap = device.tl_stream_nodemap

tl_stream_nodemap['StreamAutoNegotiatePacketSize'].value = True
tl_stream_nodemap['StreamPacketResendEnable'].value = True

##### Register the callback on the device

In [ ]:
handle = callback.register(
    device, print_buffer, now_func=datetime.now)
print(f'{TAB1}Registered \'{print_buffer.__name__}\' function '
        f'on {device}\'')

##### As stream starts it will grab buffers and pass them to the callback

In [ ]:

device.start_stream(1)
print(f'{TAB1}Stream started')

time.sleep(5)

device.stop_stream()
print(f'{TAB1}Stream stopped')

##### Deregister each handle in the handle list
> Must be called before device is destroyed

In [ ]:
callback.deregister(handle)

system.destroy_device(device)